# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770864983615                   -0.52    9.0    164ms
  2   -2.772144604657       -2.89       -1.32    1.0    120ms
  3   -2.772170343786       -4.59       -2.52    1.0    100ms
  4   -2.772170673981       -6.48       -3.37    1.0    115ms
  5   -2.772170721322       -7.32       -3.90    2.0    121ms
  6   -2.772170722959       -8.79       -5.09    1.0    107ms
  7   -2.772170723014      -10.26       -5.40    2.0    127ms
  8   -2.772170723015      -11.91       -6.22    1.0    115ms
  9   -2.772170723015      -12.82       -6.50    2.0    130ms
 10   -2.772170723015      -13.85       -7.27    1.0    115ms
 11   -2.772170723015      -14.01       -8.07    2.0    130ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770788643527                   -0.52    9.0    168ms
  2   -2.7

1.7735578873637958

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770753200787                   -0.52    9.0    168ms
  2   -2.772059601326       -2.88       -1.32    1.0    131ms
  3   -2.772082996261       -4.63       -2.44    1.0    117ms
  4   -2.772083329514       -6.48       -3.12    1.0    107ms
  5   -2.772083417047       -7.06       -4.05    2.0    123ms
  6   -2.772083417628       -9.24       -4.45    1.0    112ms
  7   -2.772083417808       -9.75       -5.48    1.0    112ms
  8   -2.772083417811      -11.64       -6.16    1.0    118ms
  9   -2.772083417811      -13.12       -6.43    2.0    133ms
 10   -2.772083417811      -13.65       -7.27    1.0    117ms
 11   -2.772083417811   +  -14.21       -8.06    2.0    133ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0